# Übung 3.5

## a) Aufstellen des erweiterten primalen und des dualen Systems
Zuerst ist das primale System wie folgt definiert:  
  
Zielfunktion mit den technologiespezifischen Grenzkosten:  
$ min \text{ }z = 92.7 \cdot P_k + 79.3 \cdot P_{GuD} + 115 \cdot P_G $ 
  
Nebenbedingungen:  
$ P_k + P_{GuD} + P_G = P_L $  
$ P_k <= 600 $  
$ P_{GuD} <= 400 $  
$ P_G <= 300 $  
$ 0.85 \cdot P_k + 0.345 \cdot P_{GuD} + 0.5 P_G <= CO_{2,Grenze} $

Die $CO_{2,Grenze} $ wird über die Summe der Erzeugung pro Kraftwerk multipliziert mit dem Emissionsfaktoren bezogen auf den Primärenergieeinsatz dividiert durch die Effizienz errechnet.

Das duale System lautet damit:  
Zielfunktion:  
$ max \text{ }w = P_L \cdot y_1 + 600 \cdot y_2 + 400 \cdot y_3 + 300 \cdot y_4 + CO_{2,Grenze} \cdot y_5 $  

Nebenbedingung:     
$ y_1 + y_2 + 0.85 \cdot y_5 >= 92.7 $  
$ y_1 + y_3 + 0.345 \cdot y_5 >= 79.3$  
$ y_1 + y_4 + 0.5 \cdot y_5 >= 115$     

Anmerkung: $P_L$ ist ein Residuallast-Profil, bei dem das Lastprofil um die Erneuerbaren reduziert wird. Daher werden die Erneuerbaren nicht mit ihren Grenzkosten berücksichtigen.

## b) Lösen des primalen erweiterten Models

In [1]:
#Einlesen der Module
from pyomo.environ import *
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


T = 24
timesteps = np.arange(T)

#Zertifikatspreis
c_CO2 = 0 # EUR/tCO2

# Marginal costs der thermischen Kraftwerke berechnen
# Werte wurden auf die Angabe angepasst
thermalPlant = ['Kohle', 'GuD', 'Gasturbine']
power = {'Kohle': 600,
         'GuD': 400,
         'Gasturbine': 300} # MW
efficiency = {'Kohle': 0.41,
         'GuD': 0.58,
         'Gasturbine': 0.4} 
fuel_price = {'Kohle': 10,
         'GuD': 30,
         'Gasturbine': 30} # EUR/MWhprim
emission_factor = {'Kohle': 0.35,
         'GuD': 0.2,
         'Gasturbine': 0.2} # tCO2/MWhprim
MC = {} # marginal costs in EUR/MWh
emissions = {} # emissions in tCO2/MWh
for n in thermalPlant:
    MC[n] = (fuel_price[n] + emission_factor[n] * c_CO2) / efficiency[n]
    emissions[n] = emission_factor[n] / efficiency[n]

# Daten laden, PV und Wind laden
df = pd.read_excel('Last_PV_Wind.xlsx')
wind = df['Wind 300 MW']
PV = df['PV 200 MW Sommer']
load = df['Last Sommer [MW]']-df['Wind 300 MW']-df['PV 200 MW Sommer'] # Gruppe 3
#print(load)
#die Residuallast (in unserem Fall mit load definiert) entspricht der gesamten Nachfrage minus der Erzeugung aus PV und minus der Erzeugung aus Wind
#Residuallast muss von thermischen Kraftwerken gedeckt werden 

# Pyomo Modell aufstellen

model = ConcreteModel()

model.x = Var(thermalPlant, timesteps, within = NonNegativeReals)

# Zielfunktion, Kosten sollen minimiert werden
#dazu werden die Grenzkosten aller Kraftwerke zu jedem timestep summiert.
model.obj = Objective(
    expr = sum(model.x[n,t] * MC[n] for n in thermalPlant for t in timesteps), 
    sense= minimize)

#Nebenbedingungen bezüglich Leistung und Last 
def power_constraint_rule(model, n, t):    
   return model.x[n,t] <= power[n]
model.power_con = Constraint(thermalPlant, 
                             timesteps, 
                             rule = power_constraint_rule)

def load_constraint_rule(model, t):    
   return sum(model.x[n,t] for n in thermalPlant) == load.loc[t]
model.load_con = Constraint(timesteps, 
                            rule = load_constraint_rule)

def emissions_constraint_rule(model, t):    
   return sum(model.x[n,t]*emissions[n] for t  in timesteps for n in thermalPlant) <= 6406.8
model.emissions_con = Constraint(timesteps, 
                            rule = emissions_constraint_rule)

###Schattenvariable für Punkt c)###
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)

opt = SolverFactory('gurobi')
opt_success = opt.solve(model)

model.display()

# get values of optimization variables
PowerThermal = pd.DataFrame(index = timesteps, columns = thermalPlant)
guDPower = np.empty(T, dtype=float)
gasPower = np.empty(T, dtype=float)
kohlePower = np.empty(T, dtype=float)
for t in timesteps:
    for n in thermalPlant:
        PowerThermal.loc[t, n] = model.x[n,t].value
        guDPower[t] = PowerThermal.loc[t, 'GuD']
        gasPower[t] = PowerThermal.loc[t, 'Gasturbine']
        kohlePower[t] = PowerThermal.loc[t, 'Kohle']

#Berechnung der minimalen Kosten und der gesamten Emissionen
print()
totalMinCost = model.obj()
print("Gesamtkosten der Stromversorgung:", totalMinCost, "EUR" )
totalEmissions = sum(model.x[n,t].value * emissions[n] for n in thermalPlant for t in timesteps)
print("Gesamte CO2 Emissionen", totalEmissions, "tCO2" )

Model unknown

  Variables:
    x : Size=72, Index={Kohle, GuD, Gasturbine}*{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
        Key                : Lower : Value               : Upper : Fixed : Stale : Domain
         ('Gasturbine', 0) :     0 :  40.325732899022796 :  None : False : False : NonNegativeReals
         ('Gasturbine', 1) :     0 :  26.319218241042336 :  None : False : False : NonNegativeReals
         ('Gasturbine', 2) :     0 :  14.136807817589556 :  None : False : False : NonNegativeReals
         ('Gasturbine', 3) :     0 :  14.885993485341999 :  None : False : False : NonNegativeReals
         ('Gasturbine', 4) :     0 :  23.654723127035822 :  None : False : False : NonNegativeReals
         ('Gasturbine', 5) :     0 :  33.306188925081415 :  None : False : False : NonNegativeReals
         ('Gasturbine', 6) :     0 :  115.29641693811072 :  None : False : False : NonNegativeReals
         ('Gasturbine', 7) :     0 :  260.26367

## c) Analyse der Schattenvariablen

In [3]:
print("Wert der Schattenvariable [tCO2h]:")
for t in timesteps:
    print("Stunde", t+1, ":", model.dual[model.emissions_con[t]])

Wert der Schattenvariable [tCO2h]:
Stunde 1 : 0.0
Stunde 2 : 0.0
Stunde 3 : 0.0
Stunde 4 : 0.0
Stunde 5 : 0.0
Stunde 6 : 0.0
Stunde 7 : 0.0
Stunde 8 : 0.0
Stunde 9 : 0.0
Stunde 10 : 0.0
Stunde 11 : 0.0
Stunde 12 : 0.0
Stunde 13 : 0.0
Stunde 14 : 0.0
Stunde 15 : 0.0
Stunde 16 : 0.0
Stunde 17 : 0.0
Stunde 18 : 0.0
Stunde 19 : 0.0
Stunde 20 : 0.0
Stunde 21 : 0.0
Stunde 22 : 0.0
Stunde 23 : 0.0
Stunde 24 : -143.10344827586206


## d) Interpretation der Schattenvariable

Die Nebenbedingung wird in den Stunden 1 - 23 erfüllt, da in diesem Zeitraum das Modell kostenminiert lösbar ist ohne die Emissionsgrenzen zu überschreiten. In der 24. Stunde würde die Emissionsgrenze erstmalig überschritten werden. Die Schattenvariable in dieser Stunde stellt die Differenz zum CO2-Zertifikatspreis dar, bei dem der kostenminimale Betrieb die Emissionsgrenze nicht überschreitet. Der CO2-Preis von 143.1 €/$t_{CO2}$ deckt sich mit dem in 3.4.c berechnetem CO2-Preis, bei dem die minimalen Emissionen auftreten. In der Klimapolitik, könnten die CO2-Preise anhand der Schattenvariable dynamisch angepasst werden, sodass der kostenminimale Kraftwerksatz auch dem CO2-minimalen entspricht.